In [52]:
## Importing the Libraries
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import csv
import pandas as pd
import sqlite3
import pymongo
from pymongo import MongoClient as PyConnection
import json
from collections import defaultdict
import csv, json, redis
import sys
import re


## Establishing connection with Redis database
REDIS_HOST = 'localhost'
connr = redis.Redis(REDIS_HOST)


## Establishing connection with MongoDB database
connect = PyConnection()
db = connect.Mixed_MongoDB
product = db.Product_catalogue
customer = db.Customer_details


## Defining the required functions
def dict_factory(cursor, row):
    d = {}
    for idx,col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def end():
    print ('\nTime elapsed:'  + str((time()-t)*1000) + ' ms')

# product.create_index([('product_name','text')])


## Establishing connection with SQLite database
connection = sqlite3.connect("Mixed_SQLite.db")
connection.row_factory = dict_factory
crsr = connection.cursor()


## Taking the input
print('\n1. Search product by product name \n2. Search product by price range', 
      '\n3. Get a sorted list of products in a price range \n4. Search product by rating', 
      '\n5. Search product by brand \n6. Search customer by customer state',
      '\n7. Search payment details by payment type \n8. Update the price of a product',
      '\n9. Insert a new customer detail \n10. Get payment details for a product',
      '\n11. Update the product hits of a product')
i = input('Enter your choice: ')


## Query execution
if (i == '1'):
    r = input('Enter the product name: ')
    t = time()
    x = "\"" + r + "\""
    result = (list(product.find({"$text": {"$search": x}})))
    end()
    print('\n')
    print(result)
    print('\n')
    print(len(result))
    
elif (i == '2'):
    l = int(input("Enter the lower limit: "))
    g = int(input("Enter the upper limit: "))
    t = time()
    result = list(product.find({"retail_price": {"$gt":  l, "$lt": g}}))
    end()
    print('\n')
    print(result)
    print('\n')
    print(len(result))
    
elif (i == '3'):
    l = int(input("Enter the lower limit: "))
    g = int(input("Enter the upper limit: "))
    t = time()
    result = list(product.find({"retail_price": {"$gt": l, "$lt": g}}).sort("retail_price", pymongo.ASCENDING))
    end()
    print('\n')
    print(result)
    print('\n')
    print(len(result))
  
elif (i == '4'):
    r = int(input('Enter the rating: '))
    t = time()
    result = list(product.find({"product_rating": r}))
    end()
#     print('\n')
#     print(result)
    print('\n')
    print(len(result))

elif (i == '5'):
    r = input('Enter the brand name: ')
    t = time()
    product.create_index([('brand', 'text')])
    result = (list(product.find({"$text": {"$search": r}})))
    end()
    print('\n')
    print(result)
    print('\n')
    print(len(result))

elif (i == '6'):
    r = input('Enter the state name: ')
    t = time()
    result = (list(customer.find({"customer_state": r})))
    end()
    print('\n')
    print(len(result))

elif (i=='7'):
    r = input('Enter the payment type: ')
    t = time()
    crsr.execute("Select * From payment Where payment_type = ?", (r,))
    result = crsr.fetchall()
    end()
#     print('\n')
#     print(result)
    print('\n')
    print(len(result))
    
elif (i =='8'):
    r = input('Enter the product id: ')
    t = time()
    product.update_many({"product_id": r}, {"$set":{"retail_price": 1000}})
    end()

elif (i == '9'):
    r = input()
    t = time()
    a = r.split(',')
    c = list(a)
    customer.insert({"customer_id":c[0], "customer_zip_code_prefix":c[1],"customer_city":c[2],"customer_state":c[3]})
    end()

elif(i=='10'):
    r = input('Enter the product name: ')
    s = input('Enter the payment value: ')
    t = time()
    x = "\"" + r + "\""
    li2 = list(product.find({"$text":{"$search": x}},{"product_id":1}))
    li10 = [d['product_id'] for d in li2]

    placeholder= '?' # For SQLite. See DBAPI paramstyle.
    placeholders= ', '.join(placeholder for unused in li10)
    query= 'Select * From Payment Where payment_value = 65.71 And product_id In (%s)' % placeholders
    crsr.execute(query, li10)
    result = crsr.fetchall()
    end()
    print('\n')
    print(result)
    print(len(result))

elif (i == '11'): 
    r = input('Enter the product name: ')
    t = time()
    x = "\"" + r + "\""
    li2 = list(product.find({"$text":{"$search": x}},{"product_id":1}))
    li10 = [d['product_id'] for d in li2]
    print(li10)
    end()
    
    s = input('Enter the product id: ')
    t = time()
    connr.incr(s,amount=1)
    end()
    
    
## Terminating the connection with SQLite database
connection.commit()
connection.close()


1. Search product by product name 
2. Search product by price range 
3. Get a sorted list of products in a price range 
4. Search product by rating 
5. Search product by brand 
6. Search customer by customer state 
7. Search payment details by payment type 
8. Update the price of a product 
9. Insert a new customer detail 
10. Get payment details for a product 
11. Update the product hits of a product
Enter your choice: 11
Enter the product name: Alisha Solid Women's Cycling Shorts
['622a3f08e882eef68dd982a5d0feee69', '5a885007d9eed9680c983b19126a4bee', 'db580783a754f97648c304e049cf4d76', 'cdad30e46a6fc84785a525ad6b5cf748', '99977819f93de86edd087f696bc75e57', '9d04b62c69bd020f5b68d1f887b7c930', 'ebc2160765952857f3bec4d6d76673e8', '17f6a6d7f6e0334b28d8d026d12db169', 'a5a7c5c2be6edf0b10ce28ba5c6b4a69', 'dc52f0f5d3ec37a93eaf956cde4e5d2c', '3731dd5544cd9646d73050e946641da8', '518ef5de2c2b3a255e326a4594ba15d9', '3bb7f144022e6732727d8d838a7b13b3', '7bb6f29c2be57716194f96496660c7c2', '8c9210